# HSMA 3C: Exercise 1 - Interactive Plots

*The first cell only needs to be run if running in colab!* 

In [ ]:
!git clone https://github.com/hsma-programme/h6_3c_interactive_plots_travel.git
%cd /content/h6_3c_interactive_plots_travel/h6_3c_interactive_plots_travel/exercises_colab

Import the required packages.

In [ ]:
import pandas as pd
import geopandas
import folium

## Core

### The Dentist Dataset - Point Data

Run this section to import the dentist activity dataframe and convert it to EPSG:4326 (which will work with Folium, unlike EPSG:27700 or EPSG:3857). 

In [ ]:
dentist_activity_dec_2023 = geopandas.read_file("../datasets/exercise_1/dentist_activity_dec_2023.geojson")
dentist_activity_dec_2023 = dentist_activity_dec_2023[dentist_activity_dec_2023['result_nhs_ha'] == "South West"]
dentist_activity_dec_2023 = dentist_activity_dec_2023.to_crs('EPSG:4326').reset_index(drop=True)
dentist_activity_dec_2023.head()

Create an empty map for us to put our dentist points onto.

In [ ]:
dentist_map_interactive = folium.Map(
    location=[55,-3.5],
    zoom_start=5.25,
    tiles='cartodbpositron'
    )

Filter out instances with no geometry.

In [ ]:
dentist_activity_dec_2023 = dentist_activity_dec_2023[~dentist_activity_dec_2023['geometry'].is_empty]

Create a geometry list from the GeoDataFrame.

In [ ]:
dentist_activity_dec_2023_geometry_list = [[point.xy[1][0], point.xy[0][0]] for point in dentist_activity_dec_2023.geometry]
dentist_activity_dec_2023_geometry_list

Loop through the points in **dentist_activity_dec_2023_geometry_list** and add them to your map as Folium markers.

In [ ]:
for coordinates in dentist_activity_dec_2023_geometry_list:
    dentist_map_interactive.add_child(
        folium.Marker(
            location=coordinates
        )
    )

Display the updated map by running the line below.

In [ ]:
dentist_map_interactive

Add a tooltip showing the practice name and the number of UDA units they delivered.

In [ ]:
for i, coordinates in enumerate(dentist_activity_dec_2023_geometry_list):
    dentist_map_interactive.add_child(
        folium.Marker(
            location=coordinates,
            tooltip=f"{dentist_activity_dec_2023['PROVIDER_NAME'][i]}: {dentist_activity_dec_2023['UDA_DELIVERED'][i]} units delivered"
        )
    )

dentist_map_interactive

### The stats19 dataset - Choropleths

The following code reads in the MSOA-level data as a geodataframe.

In [ ]:
stats19_choro_gdf = geopandas.read_file("../datasets/exercise_1/stats_19_counts_by_msoa_normalised_3857.geojson")
stats19_choro_gdf.head()

View the available columns.

In [ ]:
stats19_choro_gdf.columns

Create a base map. 

In [ ]:
stats19_map_interactive = folium.Map(
    location=[55,-3.5],
    zoom_start=5.25,
    tiles='cartodbpositron'
    )

Create a choropleth layer of a column of your choice.

In [ ]:
choropleth = folium.Choropleth(
    geo_data = stats19_choro_gdf,
    data = stats19_choro_gdf,
    columns = ['MSOA11CD','snow_collisions_2018_2022_rate'],
    key_on = 'feature.properties.MSOA11CD',
    fill_color = 'OrRd',
    fill_opacity = 0.4,
    line_weight = 0.3,
    legend_name = "Collisions in snow - of total collisions in MSOA (2018-2022)",
    highlight = True,
    smooth_factor = 0
)


Add the choropleth layer to the basemap.

In [ ]:
choropleth.add_to(stats19_map_interactive)

Display the final map.

In [ ]:
stats19_map_interactive

Add a tooltip.

In [ ]:
choropleth.geojson.add_child(
    folium.features.GeoJsonTooltip(
        ['MSOA11CD','snow_collisions_2018_2022_rate'],
        labels=True
    )
)

Display the updated map.

In [ ]:
stats19_map_interactive

Save the map to a file.

In [ ]:
stats19_map_interactive.save('stats19_snow_rate.html')

## Extension

### Return to your dentist map

Add the icon 'odontology'.

In [ ]:
odontology_icon_path = "../datasets/exercise_1/odontology.png"

dentist_map_interactive = folium.Map(
    location=[55,-3.5],
    zoom_start=5.25,
    tiles='cartodbpositron'
    )

for coordinates in dentist_activity_dec_2023_geometry_list:
    icon = folium.features.CustomIcon(
        odontology_icon_path,
        icon_size=(24,24)
    )

    dentist_map_interactive.add_child(
        folium.Marker(
            location=coordinates,
            icon=icon
        )
    )

dentist_map_interactive

Create a heatmap of dentists.

In [ ]:
from folium import plugins

dentist_map_heatmap = folium.Map(
    location=[55,-3.5],
    zoom_start=5.25,
    tiles='cartodbpositron'
    )

heat_data = [[point.xy[1][0], point.xy[0][0]] for point in dentist_activity_dec_2023.geometry]

heatmap_layer = plugins.HeatMap(
    heat_data,
    radius=10,
    blur=5
    )


heatmap_layer.add_to(dentist_map_heatmap)

dentist_map_heatmap

## Challenge: Bonus Exercises

### 1. Try out some of the plugins from the slides

In [ ]:
# No solution given

### 2. Explore what happens if you add a dataset of markers over the top of a choropleth

- Load in the indices of multiple deprivation file from here: https://research.mysociety.org/sites/imd2019/about/#data (choose **Local authority - MSOA-level file**)
- Join these to the geodataframe of MSOA boundaries below (the code just strips the stats19 data out, leaving you with just the boundaries).
- Create a basic map and then add a choropleth layer of the IMD deciles.
- Then plot your dentist dataset from the beginning of this notebook over the top.

In [ ]:
msoa_boundaries = geopandas.read_file("../datasets/exercise_1/stats_19_counts_by_msoa_normalised_3857.geojson")[['MSOA11CD', 'geometry']]
msoa_boundaries.head()

In [ ]:
msoa_imd = pd.read_csv("https://research.mysociety.org/sites/imd2019/media/data/imd2019_msoa_level_data.csv")
msoa_imd.head()

In [ ]:
msoa_imd_gdf = pd.merge(
    msoa_boundaries,
    msoa_imd,
    left_on="MSOA11CD",
    right_on="MSOAC",
    how="left"
    )

imd_map_interactive = folium.Map(
    location=[55,-3.5],
    zoom_start=5.25,
    tiles='cartodbpositron'
    )

choropleth = folium.Choropleth(
    geo_data = msoa_imd_gdf,
    data = msoa_imd_gdf,
    columns = ['MSOA11CD','MSOADECILE'],
    key_on = 'feature.properties.MSOA11CD',
    fill_color = 'OrRd',
    fill_opacity = 0.4,
    line_weight = 0.3,
    legend_name = "MSOA Decile",
    highlight = True,
    smooth_factor = 0
)

choropleth.add_to(imd_map_interactive)

choropleth.geojson.add_child(
    folium.features.GeoJsonTooltip(
        ['MSOA11CD','MSOAHOCLN', "MSOADECILE"],
        labels=True
    )
)

for coordinates in dentist_activity_dec_2023_geometry_list:
    imd_map_interactive.add_child(
        folium.Marker(
            location=coordinates
        )
    )


imd_map_interactive